# The Quadratic Assignment Problem

In [2]:
##This is an initialization cell. Run this first
import pandas as pd
import numpy as np
from itertools import product
import time
import math
import matplotlib
import matplotlib.pyplot as plt
import random

## Repository

In [3]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = []
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = []
            for i in string2:
                if i != '':
                    emptyarray.append(int(i))
            matrix.append(emptyarray)
        npmatrix = np.array(matrix) 
        return npmatrix
    return string_to_integers(final_data1),string_to_integers(final_data2)

In [4]:
#REPOSITORY

#small sized matrices(under 10x10)
matrix_size_4 = 'tai4a.csv'
matrix_size_5 = 'tai5a.csv'
matrix_size_6 = 'tai6a.csv'
matrix_size_7 = 'tai7a.csv'
matrix_size_8 = 'tai8a.csv'
matrix_size_9 = 'tai9a.csv'

#medium sized matrices(ranging from 10x10 to 30x30)
matrix_size_10 = 'tai10a.csv'
matrix_size_11 = 'tai11a.csv'
matrix_size_12 = 'tai12a.csv'
matrix_size_15 = 'chr15a.csv' 
matrix_size_20 = 'chr20a.csv'
matrix_size_26 = 'bur26a.csv'

#large sized matrices(30x30 and bigger)
matrix_size_40 = 'tai40a.csv'
matrix_size_60 = 'tai60.csv'
matrix_size_80 = 'tai80.csv'
matrix_size_256 = 'tai256c.csv'

datamatrix = CSVtoNumpyArray('tai4a.csv') # Decide the size of problem to run in the code 
                                            # (clue: the number in the original name is the size)
MatrixLoc = datamatrix[0]
MatrixFlow = datamatrix[1]

## Preliminary functions

In [5]:
def Exhaustive_search(listofpermutations,MatrixLoc,MatrixFlow):
    """
    Input:
    MatrixLoc
    MatrixFlow
    listofpermutations
    
    Output:
    The optimal permutation
    the optimal cost
    in a tuple
    """
    start = time.time()
    matrix_length = len(MatrixLoc)
    no_of_permutations = len(listofpermutations)
    arraysol = []
    
    #generate the multiples (that function we are optimising)
    for j in range(no_of_permutations):
        perm = listofpermutations[j]
        total = 0
        for i in range(matrix_length):
            for j in range(i,matrix_length):
                total += MatrixLoc[i][j]*MatrixFlow[int(perm[i])][int(perm[j])]#this is that function that 
                                                #adds the products of different combinations of factories
        arraysol.append(total)
    
    finalcost = min(arraysol)
    finalindex = np.argmin(arraysol) #finds the optimal set of locations to factories(Which I stupidly 
                                        #called flow)
    end = time.time()
    thetime = end - start    
    return listofpermutations[finalindex],finalcost,thetime

## Wolf pack algorithm
### https://www.researchgate.net/publication/275069197_Wolf_Pack_Algorithm_for_Unconstrained_Global_Optimization Wolf Pack Algorithm for Unconstrained Global Optimization Hu-Sheng Wu and Feng-Ming Zhang

### So this algorithm describes 3 specific behaviours for wolves. I have redefined them to suit the problem at hand. Read the original journal for the step functions defined for each bhaviour. Otherwise I have kept to the spirit of the algorithm:

#### Scouting:
Here wolves do medium steps around where they were placed (reasonably randomly) to find a good minimum. Here I am using a last value swap as it is low in computation weight and a good analogy of the circular motion in the original step search

#### Calling
Here wolves who scouted successfully would call others closer to the lead wolf (the minimum). These are the big steps towards a successful minimum so I will find the initial permutation number (i.e. the main branch of the lead wolf) and will swap it with the wolf in question's leaing value:
EG: if the wolf in question is (2,4,6,1,3,5) and (6,5,4,3,2,1) is the current leading wolf, then the wolf in question will become: (6,4,2,1,3,5)
This is repeated for the next 2 positions.

#### Beseiging
Here the wolves surround the lead wolf. Here I will run the 6 permutations the bottom of the branches. This will be the smallest step. The best of these move onto the next step and the cycle repeats.


In [6]:
def replenish_herd(length,populationsize):
    """
    Input:
    length is the size of the matrix
    populationsize is the number of permutations you need
    
    Output:
    listofpermutations: list of lists
    """
    listofpermutations= []
    triallist = list(range(length))
    i = 0
    for i in range(populationsize):
        random.shuffle(triallist)
        dummy = triallist[:]
        listofpermutations.append(dummy)
    return listofpermutations

In [51]:
# Find the minimum
def find_minimum(listofpermutations,MatrixLoc,MatrixFlow):
    """
    Input:
    list of permutations: a list of lists
    
    Output:
    opt perm: list
    opt perm length: float
    """
    matrix_length = len(listofpermutations[0])
    arraysol = []
    #generate the multiples (that function we are optimising)
    for j in listofpermutations:
        total = 0
        for i in range(matrix_length):
            for k in range(matrix_length):
                if i!=k:
                    total += MatrixLoc[i][k]*MatrixFlow[int(j[i])][int(j[k])]
        arraysol.append(total)
    finalcost = min(arraysol)
    finalindex = np.argmin(arraysol) #finds the optimal set of locations to factories(Which I stupidly 
                                        #called flow)
    return finalcost,finalindex,listofpermutations[finalindex]


In [130]:
# Find the minimum
def find_maximum(listofpermutations,MatrixLoc,MatrixFlow):
    """
    Input:
    list of permutations: a list of lists
    
    Output:
    opt perm: list
    opt perm length: float
    """
    matrix_length = len(listofpermutations[0])
    arraysol = []
    #generate the multiples (that function we are optimising)
    for j in listofpermutations:
        total = 0
        for i in range(matrix_length):
            for k in range(matrix_length):
                if i!=k:
                    total += MatrixLoc[i][k]*MatrixFlow[int(j[i])][int(j[k])]
        arraysol.append(total)
    finalcost = max(arraysol)
    finalindex = np.argmax(arraysol) #finds the optimal set of locations to factories(Which I stupidly 
                                        #called flow)
    return finalcost,finalindex,listofpermutations[finalindex]

In [9]:
def scout(listofpermutations):
    """
    Input:
    listofpermutations: list of lists
    
    Output:
    newlistofpermutations: list of lists
    """
    #All this does is swap the last two positions
    for i in listofpermutations:
        newlast = i[-2]
        oldlast = i[-1]
        i[-1] = newlast
        i[-2] = oldlast
        
    return listofpermutations

In [10]:
def call(listofpermutations,minindex):
    """
    Input:
    listofpermutations: list of lists
    minindex: int object with the position of the min list in the list
    
    Output:
    newlistofpermutations: list of lists
    """
    keybranch = listofpermutations[minindex][0]
    for i in listofpermutations:
        keyindex = i.index(keybranch)
        newfirst = i[keyindex]
        oldfirst = i[0]
        i[0] = newfirst
        i[keyindex] = oldfirst
    return listofpermutations
    

In [11]:
def ourpermutations(iterable, r=None):
    """
    Input:
    String or numbers separated by a space
    optional= the length that the permutations must be
    
    Output:
    a generator of permutations
    """
    
    pool = iterable
    n = len(pool)
    r = n if r is None else r
    for indices in product(range(n), repeat=r):
        if len(set(indices)) == r:
            yield list(pool[i] for i in indices)

In [12]:
def beseige(listofpermutations):
    #don't forget to catch duplicates before the next search
    """
    Input:
    listofpermutations: list of lists
    
    Output:
    newlistofpermutations: list of lists
    """
    newlistofpermutations = []
    for i in listofpermutations:
        ends = list(ourpermutations(i[-3:]))
        for j in ends:
            temp = i[:-3]
            temp= temp + j
            newlistofpermutations.append(temp)
    
    return newlistofpermutations

In [56]:
#rank post-beseige and return desired amount
def rankperms(listofpermutations,noofbest,MatrixLoc,MatrixFlow):
    """
    Input:
    list of permutations: a list of lists
    
    Output:
    opt perm: list
    opt perm length: float
    """
    matrix_length = len(listofpermutations[0])
    arraysol = []
    
    #generate the multiples (that function we are optimising)
    for j in listofpermutations:
        total = 0
        for i in range(matrix_length):
            for k in range(matrix_length):
                if i != k:
                    total += MatrixLoc[i][k]*MatrixFlow[int(j[i])][int(j[k])]
        arraysol.append(total) 
    finalindices = np.argsort(arraysol) #ranks the options
    
    #takes best number
    bestperms = []
    for i in range(noofbest):
        bestperms.append(listofpermutations[finalindices[i]])
    
    return bestperms


In [164]:
def wolfpack(MatrixLoc,MatrixFlow,Totaliterations,packsize,callweight,noofbest):
    """
    Input:
    MatrixLoc: input data numpy matrix
    MatrixFlow: input data numpy matrix
    Totaliterations: int of number of times we run the code (~100)
    packsize: int of number wolves each time(~100)
    callweight: int number of times we call the wolves(~3)
    noofbest: int number of values moving onto the next replenishment(1<noofbest<packsize/2)
    
    
    Output:
    optimal weight: float
    optimal permutation: list
    """
    #initial values
    listofpermutations = []
    length = len(MatrixLoc)
    

    for i in range(Totaliterations):
        #filling up the herd
        listofpermutations += replenish_herd(length,packsize)
        minimumperm = find_minimum(listofpermutations,MatrixLoc,MatrixFlow)[2]
        
        #scout
        listofscouted = scout(listofpermutations)
        listofscouted.append(minimumperm)
        minimumindex = find_minimum(listofscouted,MatrixLoc,MatrixFlow)[1]
        
        #call
        listofcalled = listofscouted
        for i in range(callweight):
            listofcalled = call(listofcalled,minimumindex)
            minimumindex = find_minimum(listofcalled,MatrixLoc,MatrixFlow)[1]
        
        #beseige
        listofbeseige = beseige(listofcalled)
        listofpermutations = rankperms(listofbeseige,noofbest,MatrixLoc,MatrixFlow)
        listofpermutations.append(minimumperm)
        
    return  find_minimum(listofpermutations,MatrixLoc,MatrixFlow)[::2]
        

In [165]:
Totaliterations = 100
packsize = 40
callweight = 3
noofbest = packsize//3
wolfpack(MatrixLoc,MatrixFlow,Totaliterations,packsize,callweight,noofbest)


(142502, [3, 0, 7, 5, 9, 1, 8, 2, 6, 4])

In [77]:
#grid search for optimal values
def wolfpackcomparisons(test):
    """
    Output:
    List of solutions
    solution,iteration number,packsize
    """
    wolfsolutions = []
    
    for q in range(9,12):
        datamatrix = CSVtoNumpyArray('tai' + str(q) +'a.csv')
        MatrixLoc = datamatrix[0]
        MatrixFlow = datamatrix[1]
        tempsolutions = []
        #print(q,MatrixFlow)
        for r in range(3,11):
            for s in range(11,13):
                tempsol = [wolfpack(MatrixLoc,MatrixFlow,r*10,q*s*2,3,s*5//3),r,s]
                #print(tempsol)
                tempsolutions.append(tempsol)
        wolfsolutions.append(tempsolutions)

        
    return wolfsolutions

In [78]:
soldata = wolfpackcomparisons(True)


In [79]:
soldata

[[[(2282, [2, 5, 8, 7, 6, 3, 4, 1, 0]), 3, 11],
  [(2306, [3, 4, 0, 1, 2, 5, 8, 6, 7]), 3, 12],
  [(2308, [2, 0, 4, 7, 3, 6, 8, 5, 1]), 4, 11],
  [(2298, [0, 6, 7, 4, 3, 8, 5, 1, 2]), 4, 12],
  [(2286, [3, 4, 0, 1, 2, 8, 5, 6, 7]), 5, 11],
  [(2316, [4, 3, 0, 1, 2, 8, 5, 6, 7]), 5, 12],
  [(2328, [8, 6, 7, 4, 3, 0, 2, 1, 5]), 6, 11],
  [(2282, [2, 5, 8, 7, 6, 3, 4, 1, 0]), 6, 12],
  [(2330, [5, 1, 2, 4, 0, 3, 7, 6, 8]), 7, 11],
  [(2306, [3, 4, 0, 1, 2, 5, 8, 6, 7]), 7, 12],
  [(2282, [2, 5, 8, 7, 6, 3, 4, 1, 0]), 8, 11],
  [(2282, [2, 5, 8, 7, 6, 3, 4, 1, 0]), 8, 12],
  [(2282, [2, 5, 8, 7, 6, 3, 4, 1, 0]), 9, 11],
  [(2282, [2, 5, 8, 7, 6, 3, 4, 1, 0]), 9, 12],
  [(2282, [2, 5, 8, 7, 6, 3, 4, 1, 0]), 10, 11],
  [(2382, [5, 0, 8, 7, 6, 3, 4, 1, 2]), 10, 12]],
 [[(152006, [5, 7, 6, 4, 0, 2, 9, 8, 3, 1]), 3, 11],
  [(152504, [3, 7, 6, 4, 1, 9, 8, 2, 0, 5]), 3, 12],
  [(140122, [8, 0, 7, 4, 9, 1, 3, 2, 6, 5]), 4, 11],
  [(150030, [0, 8, 4, 5, 2, 7, 3, 9, 1, 6]), 4, 12],
  [(141458, [3, 0

In [ ]:
([(['8', '0', '6', '4', '9', '10', '5', '2', '7', '1', '3'],   190016,   3786.5291616916656)],
 [(['8', '0', '6', '4', '9', '10', '5', '2', '7', '1', '3'],   190016,  3029.7006783485413)], True)

In [171]:
def wolfpack30test(start,end,minimum,Totaliterations,packsize):
    """
    Input:
    start = int start length
    end = int final length +1
    minimum,Totaliterations,packsize,callweight = lists of required values
    
    Output:
    percentage successful
    """
    
    tot = []
    totalindex = 0
    
    for q in range(start,end):
        wolfsolutions = []
        datamatrix = CSVtoNumpyArray('tai' + str(q) +'a.csv')
        MatrixLoc = datamatrix[0]
        MatrixFlow = datamatrix[1]
        tempsolutions = []
        #print(q,MatrixFlow)
        for r in range(30):
            tempsol = [wolfpack(MatrixLoc,MatrixFlow,Totaliterations[q-4],packsize[q-4],3,packsize[q-4]//3),r,q]
            tempsolutions.append(tempsol)
        wolfsolutions.append(tempsolutions)
    
        total = 0
        for i in wolfsolutions[0]:
            if i[0][0] == minimum[totalindex]:
                total+=1
        tot.append(total/30 *100)
        totalindex += 1


        
    return tot

In [ ]:
ans = wolfpack30test(4,12,[790,628,626,1478,1808,2282,135028,190016],[10,30,50,50,100,200,500,500],
                     [10,10,20,20,50,100,200,200])
ans

In [100]:
ans[0]#[0][0]

[(824, [3, 2, 1, 0]), 0, 4]

In [127]:
print(math.factorial(11),11**2)


39916800 121


In [166]:
def wolfpack_evolution(MatrixLoc,MatrixFlow,Totaliterations,packsize,callweight,noofbest):
    """
    Input:
    MatrixLoc: input data numpy matrix
    MatrixFlow: input data numpy matrix
    Totaliterations: int of number of times we run the code (~100)
    packsize: int of number wolves each time(~100)
    callweight: int number of times we call the wolves(~3)
    noofbest: int number of values moving onto the next replenishment(1<noofbest<packsize/2)
    
    
    Output:
    optimal weight: float
    optimal permutation: list
    """
    #initial values
    listofpermutations = []
    length = len(MatrixLoc)
    evolution = []

    for i in range(Totaliterations):
        #filling up the herd
        listofpermutations += replenish_herd(length,packsize)
        minimumperm = find_minimum(listofpermutations,MatrixLoc,MatrixFlow)[2]
        
        #scout
        listofscouted = scout(listofpermutations)
        listofscouted.append(minimumperm)
        minimumindex = find_minimum(listofscouted,MatrixLoc,MatrixFlow)[1]
        
        #call
        listofcalled = listofscouted
        for i in range(callweight):
            listofcalled = call(listofcalled,minimumindex)
            minimumindex = find_minimum(listofcalled,MatrixLoc,MatrixFlow)[1]
        
        #beseige
        listofbeseige = beseige(listofcalled)
        listofpermutations = rankperms(listofbeseige,noofbest,MatrixLoc,MatrixFlow)
        listofpermutations.append(minimumperm)
        
        #evolution
        minimum = find_minimum(listofpermutations,MatrixLoc,MatrixFlow)[0]
        maximum = find_maximum(listofpermutations,MatrixLoc,MatrixFlow)[0]
        evolution.append([minimum,maximum,i])
        
    return  find_minimum(listofpermutations,MatrixLoc,MatrixFlow)[::2],evolution
        

In [167]:
datamatrix = CSVtoNumpyArray('tai10a.csv')
MatrixLoc = datamatrix[0]
MatrixFlow = datamatrix[1]
ev = wolfpack_evolution(MatrixLoc,MatrixFlow,100,15,3,4)
ev

((144314, [7, 9, 4, 3, 5, 0, 8, 1, 2, 6]),
 [[167714, 185820, 2],
  [156982, 159122, 2],
  [156982, 193486, 2],
  [156982, 193486, 2],
  [156982, 193486, 2],
  [144314, 193486, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178340, 2],
  [144314, 178

## Random random 

In [541]:
math.factorial(7)

5040

In [542]:
50//3

16

In [41]:
perm = [3, 5, 1, 2, 8, 0, 6, 10, 7, 4, 11, 9]
perm= [8,1,6,2,11,10,3,5,9,7,12,4]
perm = [9,4,6,3,11,7,12,2,8,10,1,5]
#perm = [3,20,7,18,9,12,19,4,10,11,1,6,15,8,2,5,14,16,13,17]
perm = [perm[i] - 1 for i in range(len(perm))]
matrix_length = len(perm)
total = 0
for i in range(matrix_length):
    for j in range(matrix_length):
        if i!=j:
            print(i,j,MatrixLoc[i][j],perm[i],perm[j],MatrixFlow[int(perm[i])][int(perm[j])])
            total += MatrixLoc[i][j]*MatrixFlow[int(perm[i])][int(perm[j])]
total

0 1 0 8 3 30
0 2 0 8 5 0
0 3 255 8 2 9
0 4 293 8 10 0
0 5 328 8 6 0
0 6 331 8 11 0
0 7 318 8 1 10710
0 8 362 8 7 0
0 9 221 8 9 867
0 10 314 8 0 2911
0 11 390 8 4 5399
1 0 0 3 8 0
1 2 1 3 5 1
1 3 255 3 2 62
1 4 293 3 10 490
1 5 329 3 6 0
1 6 331 3 11 0
1 7 318 3 1 0
1 8 363 3 7 2491
1 9 222 3 9 0
1 10 314 3 0 27
1 11 390 3 4 0
2 0 0 5 8 41986
2 1 1 5 3 0
2 3 255 5 2 7
2 4 293 5 10 5
2 5 328 5 6 9
2 6 330 5 11 0
2 7 318 5 1 3797
2 8 362 5 7 0
2 9 221 5 9 19780
2 10 313 5 0 0
2 11 390 5 4 0
3 0 255 2 8 0
3 1 255 2 3 0
3 2 255 2 5 0
3 4 93 2 10 0
3 5 120 2 6 1
3 6 106 2 11 0
3 7 106 2 1 0
3 8 113 2 7 0
3 9 81 2 9 0
3 10 63 2 0 25642
3 11 195 2 4 2
4 0 293 10 8 5
4 1 293 10 3 0
4 2 293 10 5 2791
4 3 93 10 2 0
4 5 35 10 6 0
4 6 42 10 11 0
4 7 25 10 1 1
4 8 95 10 7 0
4 9 72 10 9 7632
4 10 75 10 0 0
4 11 107 10 4 12262
5 0 328 6 8 0
5 1 329 6 3 93
5 2 328 6 5 5
5 3 120 6 2 1
5 4 35 6 10 39
5 6 24 6 11 759
5 7 13 6 1 14543
5 8 82 6 7 3720
5 9 107 6 9 2
5 10 82 6 0 948
5 11 75 6 4 0
6 0 331 11 8

39464925

In [33]:
19770580*2

39541160

In [35]:
MatrixFlow

array([[    0,     0,     5,     1,     1, 14261,   246, 55342,     0,
            1,    36,  1828],
       [    1,     0, 14568,  1064,     4,     0, 58098,  6273,     2,
            3,     0,    57],
       [25642,     0,     0,     0,     2,     0,     1,     0,     0,
            0,     0,     0],
       [   27,     0,    62,     0,     0,     1,     0,  2491,     0,
            0,   490,     0],
       [    0,    57,     0,     0,     0,     0,   456,  9591,     0,
            0,  1036,     0],
       [    0,  3797,     7,     0,     0,     0,     9,     0, 41986,
        19780,     5,     0],
       [  948, 14543,     1,    93,     0,     5,     0,  3720,     0,
            2,    39,   759],
       [    0,     0,     0,     8,     0,     2,  3516,     0,     0,
         2106,     0, 20364],
       [ 2911, 10710,     9,    30,  5399,     0,     0,     0,     0,
          867,     0,     0],
       [    0,     3, 47029,   264,  4880,     1,  5417,     0,     0,
            0,     0